In [66]:
import _csv
import math
import re
import gzip
import json
import numpy as np
import cohere
from cohere.classify import Example
co = cohere.Client('Odj4ClpHI3fobuB9bErJkfzlszJwQ018Z99gUAyE')

In [67]:
def print_classifications(response, data=True):
    for i in range(len(response.classifications)):
        prediction = response.classifications[i].prediction
        confidence = response.classifications[i].confidence
        if data:
            data = response.classifications[i].input
            print(f'Input: {data}')
        print(f'Class: {prediction}, Confidence: {confidence} \n')

In [68]:
user_prompt1 = """
My son claims that hes been having extremely violent thoughts and dreams. Not violent like he's hurting someone, but violent thoughts like horrible things happening to his loved ones. He explained one of his dreams the other day and it was so violent it was sickening. It was far beyond anything in a horror movie, he says he can't help these thoughts they just pop up. Please help!
"""
user_prompt = """
I went to the gym last night and saw many people with good fitness. However I am fat and obese. So I couldn't stand being there and came home while crying
"""

In [72]:
prompt = f"""
Break the paragraph into small sentences {user_prompt}
"""

summary = co.generate(
    model='command-xlarge-nightly',
    prompt= prompt,
    max_tokens=100,
    temperature=0.9,
    stop_sequences=["--"])
print(summary)

[cohere.Generation {
	id: d114adfe-ebe6-49ee-ad04-3a3d0f6e315f
	text: I went to the gym last night and saw many people with good fitness. But I am fat and obese, so I couldn't stand being there and came home while crying.
	likelihood: None
	token_likelihoods: None
}]


In [70]:
cut_user_prompt = re.split(r'[,.!?-]', user_prompt)
print(cut_user_prompt)

response = co.classify(
  model='0c667180-ef42-4c4d-aee8-e6fa29177e43-ft',
  inputs=cut_user_prompt)

print_classifications(response)

['\nI went to the gym last night and saw many people with good fitness', ' However I am fat and obese', " So I couldn't stand being there and came home while crying\n"]
Input: 
I went to the gym last night and saw many people with good fitness
Class: 1, Confidence: 0.7628686 

Input:  However I am fat and obese
Class: 3, Confidence: 0.735269 

Input:  So I couldn't stand being there and came home while crying

Class: 0, Confidence: 0.5047846 

